<a href="https://colab.research.google.com/github/hbgit/walletstockanalysis/blob/main/ReviewMyWalletStonks_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
import json
# Import the plotting library
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px

In [26]:
#PRIO3 = yf.Ticker("PRIO3.SA")
PRIO3 = yf.Ticker("VOO")
print(PRIO3.info)

{'exchange': 'PCX', 'shortName': 'Vanguard S&P 500 ETF', 'longName': 'Vanguard 500 Index Fund', 'exchangeTimezoneName': 'America/New_York', 'exchangeTimezoneShortName': 'EDT', 'isEsgPopulated': False, 'gmtOffSetMilliseconds': '-14400000', 'quoteType': 'ETF', 'symbol': 'VOO', 'messageBoardId': 'finmb_28117396', 'market': 'us_market', 'annualHoldingsTurnover': None, 'enterpriseToRevenue': None, 'beta3Year': 1, 'profitMargins': None, 'enterpriseToEbitda': None, '52WeekChange': None, 'morningStarRiskRating': None, 'forwardEps': None, 'revenueQuarterlyGrowth': None, 'fundInceptionDate': 1283817600, 'annualReportExpenseRatio': None, 'totalAssets': 753852547072, 'bookValue': None, 'fundFamily': 'Vanguard', 'lastFiscalYearEnd': None, 'netIncomeToCommon': None, 'trailingEps': None, 'lastDividendValue': None, 'SandP52WeekChange': None, 'priceToBook': None, 'nextFiscalYearEnd': None, 'yield': 0.0177, 'mostRecentQuarter': None, 'enterpriseValue': None, 'priceHint': 2, 'threeYearAverageReturn': 0.0

**Stock Wallet**

In [27]:
ticket_company_shares = ['ABBV','PEP','PLD','KO','STOR','PRU','MSFT','VZ','T','GOOGL','AMZN']

In [28]:
#earningsGrowth; yield; totalRevenue
metric_company_shares = [
    'marketCap',
    'revenueGrowth',
    'pegRatio',
    'returnOnAssets',
    'returnOnEquity',
    'profitMargins',
    'currentRatio',
    'debtToEquity',
    'payoutRatio',
    'recommendationMean',
    'recommendationKey',
    'totalDebt',
    'sector',
    'dividendYield',
    'averageVolume',
    'totalRevenue'
]

In [29]:
df = pd.DataFrame(index=ticket_company_shares,columns=metric_company_shares)

In [30]:
def get_fundamental_data(df):
  for symbol in df.index:
    get_sym_info = yf.Ticker(symbol)
    print("Getting symbol: " + symbol)    
    for metric in df.columns:        
      get_sym_info.info[metric]
      df.loc[symbol,metric] = get_sym_info.info[metric]    
  return df
      

In [31]:
# Defining rules to check fundamentals
# gt -> great than >= 
rulesFundmentals_gt_company_shares = {
    'marketCap' : 200000000000,
    'revenueGrowth' : 0.052,
    'pegRatio' : 2.0,
    'returnOnAssets' : 0.08000,
    'returnOnEquity' : 0.20000,
    'profitMargins' : 0.11282,
    'currentRatio' : 1.0,
    'debtToEquity' : 11.00,   
    'averageVolume' : 1700000
}

In [32]:
df = get_fundamental_data(df)

Getting symbol: ABBV
Getting symbol: PEP
Getting symbol: PLD
Getting symbol: KO
Getting symbol: STOR
Getting symbol: PRU
Getting symbol: MSFT
Getting symbol: VZ
Getting symbol: T
Getting symbol: GOOGL
Getting symbol: AMZN


In [33]:
# Print Bar Chart
def print_bar_chart_from_json(json_file):
  print(json_file)
  #s = pd.Series(json_file, name='DateValue')
  df = pd.DataFrame(json_file.items(), columns=['Ticket', 'Value'])
  fig = px.bar(df, x='Ticket', y='Value',
             color='Value',
             labels={'pop':'population of Canada'}, height=400)
  fig.show()


In [34]:
def check_peformace_last_months(list_symbol):
    # Collect data from the last 6 months    
    data = yf.download(list_symbol,period = "6mo")['Adj Close']
    flag_series = False
   
    if type(data) is pd.Series:
      flag_series = True
      single_symbol = list_symbol[0]
      data = pd.DataFrame({'Date':data.index, single_symbol:data.values})
      #set column as index
      data = data.set_index('Date')

    # https://capitalresearch.com.br/blog/calcular-rentabilidade/
    # Return  
    dict_return = {}  
    for symbol in data.columns:
      if not pd.isna(data[symbol].iloc[0]):
        dict_return[symbol] = ((data[symbol].iloc[-1]/data[symbol].iloc[0])*100)-100
      else:
        dict_return[symbol] = ((data[symbol].iloc[-1]/data[symbol].iloc[1])*100)-100

    worst_return = {}
    flag_has_bad_return = False
    for e in dict_return.keys():
      if dict_return[e] <= -20.00:
        flag_has_bad_return = True
        worst_return[e] = dict_return[e]
    
    #print(json.dumps(worst_return, sort_keys=True, indent=4))
    if flag_has_bad_return:
      print_bar_chart_from_json(worst_return)

    # ----------------- Plot all the close prices  ----------------- 
    # Pandas dataframe. pct_change() function calculates the percentage change between the current and a prior element.
    # Return cumulative product over a DataFrame or Series axis.
    # The next step is to calculate the simple daily returns of the stocks. The formula to calculate this rate is to divide each day by the previous day minus 1.   
    df = (data.pct_change()+1).cumprod()
    if not flag_series:
      df = df.reset_index()
      fig = px.line(df, x="Date", y=df.columns)
      fig.show()
    else:      
      df = df.reset_index()
      fig = px.line(df, x="Date", y=single_symbol)
      fig.show()
    
    

In [35]:
def check_fundamentals_data_shares(df, dict_rules_fundamentals):  
  
  rules_broken_by_symbol = []
  metric_broken_by_symbol = {}

  for symbol in df.index:
    count_broken_rules = 0
    tmp_metrics = []         
    
    for metric in df.columns:                                               
      if metric in dict_rules_fundamentals:                        
        if df.loc[symbol,metric] is not None:                        
          if df.loc[symbol,metric] < dict_rules_fundamentals[metric]:
            count_broken_rules += 1 
            tmp_metrics.append(df.loc[symbol,metric])
            tmp_metrics.append(dict_rules_fundamentals[metric])              
            metric_broken_by_symbol[symbol] = tmp_metrics
          if metric == "payoutRatio":
            if df.loc[symbol,metric] > 1.0:
              count_broken_rules += 1
              tmp_metrics.append(df.loc[symbol,metric])
              tmp_metrics.append(dict_rules_fundamentals[metric])              
              metric_broken_by_symbol[symbol] = tmp_metrics
          if metric == "recommendationKey":
            if df.loc[symbol,metric] ==  "sell":
              count_broken_rules += 1
              tmp_metrics.append(df.loc[symbol,metric])
              tmp_metrics.append(dict_rules_fundamentals[metric])              
              metric_broken_by_symbol[symbol] = tmp_metrics
            elif df.loc[symbol,metric] ==  "hold":
              if df.loc[symbol,"recommendationMean"] > 3.5:
                count_broken_rules += 1
                tmp_metrics.append(df.loc[symbol,"recommendationMean"])
                tmp_metrics.append(3.6)   

      if count_broken_rules >= 3: # number of rules that was broken        
        rules_broken_by_symbol.append(symbol)
      else:
        if symbol in metric_broken_by_symbol.keys():
          metric_broken_by_symbol.pop(symbol) 
    
    #try:      
    #except Exception as e:
    #  print (symbol, 'not found')

  # Useful data
  print("Stocks that broken the rules: ")  
  #print(json.dumps(metric_broken_by_symbol, sort_keys=True, indent=4))
  check_peformace_last_months(rules_broken_by_symbol)


In [36]:
check_fundamentals_data_shares(df, rulesFundmentals_gt_company_shares)

Stocks that broken the rules: 
[*********************100%***********************]  7 of 7 completed
{'PLD': -39.96076566634841, 'VZ': -35.78439948958919}


In [37]:
# ETFs analysis

In [38]:
def check_fundamentals_data_etfs(df, dict_rules_fundamentals):  
  
  rules_broken_by_symbol = []
  metric_broken_by_symbol = {}

  for symbol in df.index:
    count_broken_rules = 0
    tmp_metrics = []         
    
    for metric in df.columns:                                               
      if metric in dict_rules_fundamentals:                        
        if df.loc[symbol,metric] is not None:                        
          if df.loc[symbol,metric] < dict_rules_fundamentals[metric]:
            count_broken_rules += 1 
            tmp_metrics.append(df.loc[symbol,metric])
            tmp_metrics.append(dict_rules_fundamentals[metric])              
            metric_broken_by_symbol[symbol] = tmp_metrics
          if metric == "beta3Year":
            if df.loc[symbol,metric] > 1.2:
              count_broken_rules += 1
              tmp_metrics.append(df.loc[symbol,metric])
              tmp_metrics.append(dict_rules_fundamentals[metric])
              metric_broken_by_symbol[symbol] = tmp_metrics
      if count_broken_rules >= 3: # number of rules that was broken        
        rules_broken_by_symbol.append(symbol)
      else:
        if symbol in metric_broken_by_symbol.keys():
          metric_broken_by_symbol.pop(symbol) 
    
    #try:      
    #except Exception as e:
    #  print (symbol, 'not found')

  # Useful data
  print("Stocks that broken the rules: ") 
  #print(metric_broken_by_symbol)
  #print(json.dumps(metric_broken_by_symbol, sort_keys=True, indent=4))
  check_peformace_last_months(rules_broken_by_symbol)

In [39]:
# https://www.investopedia.com/terms/b/beta.asp
# Note pegratio and others were removed
# they have none values
metric_ETFs = [
    'quoteType',
    'beta3Year',
    'trailingAnnualDividendYield',
    'yield', 
    'marketCap', 
    'totalAssets',
    'averageVolume',
    'category'
]

In [40]:
# Defining rules to check fundamentals
# gt -> great than >= 
rulesFundmentals_gt_etfs = {
    'beta3Year' : 1.1,
    'trailingAnnualDividendYield' : 0.011,
    'yield' : 0.011,
    'totalAssets' : 4730926080,
    'averageVolume' : 621942
}

In [41]:
ticket_etfs_shares = ['VOO', 'VNQ', 'CIBR']

In [42]:
df_etfs = pd.DataFrame(index=ticket_etfs_shares,columns=metric_ETFs)

In [43]:
df_etfs = get_fundamental_data(df_etfs)

Getting symbol: VOO
Getting symbol: VNQ
Getting symbol: CIBR


In [ ]:
df_etfs

In [ ]:
check_fundamentals_data_etfs(df_etfs, rulesFundmentals_gt_etfs)

In [ ]:
# https://kernc.github.io/backtesting.py/doc/examples/Parameter%20Heatmap.html
# https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec
# 